## Reading the CSV file in spark dataframe

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SF fire calls study").getOrCreate()

In [6]:
df = spark.read.csv("./sf-fire-calls.csv", header=True, inferSchema=True)

In [5]:
df.show()

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|      UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+

24/01/22 13:27:44 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


# <span style="color:red">Question 1:</span> What were the different types of fire calls in 2018

In [59]:
df_call_type = df.select("CallType").show()
df_call_type

+----------------+
|        CallType|
+----------------+
|  Structure Fire|
|Medical Incident|
|Medical Incident|
|    Vehicle Fire|
|          Alarms|
|  Structure Fire|
|          Alarms|
|          Alarms|
|Medical Incident|
|Medical Incident|
|Medical Incident|
|  Structure Fire|
|Medical Incident|
|Medical Incident|
|  Structure Fire|
|  Structure Fire|
|  Structure Fire|
|Medical Incident|
|Medical Incident|
|Medical Incident|
+----------------+
only showing top 20 rows



In [82]:
from pyspark.sql.functions import to_date,month
from pyspark.sql.functions import year

df_with_year = df.withColumn("CallDate", to_date(df["CallDate"], "MM/dd/yyyy"))
#df_with_year.select("CallDate").distinct().show()

df_with_year = df_with_year.withColumn("CallYear", year(df_with_year["CallDate"]))
#df_with_year.select("CallYear").show()

df_with_year = df_with_year.filter(df_with_year["CallYear"] == 2018)

In [73]:
df_with_year.select("CallType").distinct().show()

+--------------------+
|            CallType|
+--------------------+
|Elevator / Escala...|
|              Alarms|
|Odor (Strange / U...|
|Citizen Assist / ...|
|              HazMat|
|        Vehicle Fire|
|               Other|
|        Outside Fire|
|   Traffic Collision|
|       Assist Police|
|Gas Leak (Natural...|
|        Water Rescue|
|   Electrical Hazard|
|      Structure Fire|
|    Medical Incident|
|          Fuel Spill|
|Smoke Investigati...|
|Train / Rail Inci...|
|           Explosion|
|  Suspicious Package|
+--------------------+



# <span style="color:red">Question 2:</span> What month in 2018 saw the highest number of fire calls

In [113]:
from pyspark.sql.functions import to_date,month
from pyspark.sql.functions import *

In [114]:
df_month_fire_calls = df_with_year.groupBy(month("CallDate").alias("month")).agg(count("UnitID").alias("sum_fire_calls"))

In [115]:
df_month_fire_calls.printSchema()

root
 |-- month: integer (nullable = true)
 |-- sum_fire_calls: long (nullable = false)



In [116]:
df_month_fire_calls.sort(desc("sum_fire_calls")).show()

+-----+--------------+
|month|sum_fire_calls|
+-----+--------------+
|   10|          1068|
|    5|          1047|
|    3|          1029|
|    8|          1021|
|    1|          1007|
|    6|           974|
|    7|           974|
|    9|           951|
|    4|           947|
|    2|           919|
|   11|           199|
+-----+--------------+



# <span style="color:red">Question 3:</span> Which neighborhodd in SF generated the most number of fire calls in 2018

In [120]:
df_Neighborhood_fire_calls = df_with_year.groupBy("Neighborhood").agg(count("UnitID").alias("most_fire_calls"))
df_Neighborhood_fire_calls.printSchema()

root
 |-- Neighborhood: string (nullable = true)
 |-- most_fire_calls: long (nullable = false)



In [124]:
df_Neighborhood_fire_calls.sort(desc("most_fire_calls")).show()

+--------------------+---------------+
|        Neighborhood|most_fire_calls|
+--------------------+---------------+
|          Tenderloin|           1393|
|     South of Market|           1053|
|             Mission|            913|
|Financial Distric...|            772|
|Bayview Hunters P...|            522|
|    Western Addition|            352|
|     Sunset/Parkside|            346|
|            Nob Hill|            295|
|        Hayes Valley|            291|
|      Outer Richmond|            262|
| Castro/Upper Market|            251|
|         North Beach|            231|
|           Excelsior|            212|
|  West of Twin Peaks|            210|
|        Potrero Hill|            210|
|           Chinatown|            191|
|     Pacific Heights|            191|
|              Marina|            191|
|         Mission Bay|            178|
|      Bernal Heights|            170|
+--------------------+---------------+
only showing top 20 rows



In [ ]:
df_Neighborhood_fire_calls

In [132]:
df_Neighborhood_fire_calls.where(df_Neighborhood_fire_calls.Neighborhood=='Presidio').show() 

+------------+---------------+
|Neighborhood|most_fire_calls|
+------------+---------------+
|    Presidio|             69|
+------------+---------------+



# <span style="color:red">Question 4:</span> Which neighborhodd in SF had the worst response time for fire calls in 2018

In [129]:
df_Delayed_fire_calls = df_with_year.groupBy("Neighborhood").agg(mean("Delay").alias("Delayed_fire_calls"))
df_Delayed_fire_calls.printSchema()

root
 |-- Neighborhood: string (nullable = true)
 |-- Delayed_fire_calls: double (nullable = true)



In [130]:
df_Delayed_fire_calls.sort(desc("Delayed_fire_calls")).show()

+--------------------+------------------+
|        Neighborhood|Delayed_fire_calls|
+--------------------+------------------+
|           Chinatown| 6.190314097905762|
|            Presidio|5.8292270414492755|
|     Treasure Island|      5.4537037125|
|        McLaren Park| 4.744047642857142|
|Bayview Hunters P...|4.6205619568773955|
|    Presidio Heights| 4.594131472394366|
|        Inner Sunset| 4.438095199935065|
|      Inner Richmond| 4.364728682713178|
|Financial Distric...| 4.344084618290156|
|      Haight Ashbury| 4.266428599285714|
|            Seacliff| 4.261111146666666|
|  West of Twin Peaks| 4.190952390857142|
|        Potrero Hill| 4.190555557428572|
|     Pacific Heights| 4.180453718900524|
|          Tenderloin| 4.101519516597271|
|Oceanview/Merced/...| 3.947242180719425|
|           Excelsior|3.9363993797169807|
|         North Beach|  3.88924964034632|
|           Lakeshore|3.8815513650943387|
|         Mission Bay| 3.854868952191012|
+--------------------+------------